# Logs SIRETISATION

In [ ]:
import os, glob, shutil, json
os.chdir('../')
current_dir = os.getcwd()
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
from inpi_insee import preparation_data
path = os.getcwd()
parent_path = str(Path(path).parent)

In [ ]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_athena import service_athena
bucket = 'calfdata'
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata') 

In [ ]:
path ="INPI/TC_1/03_siretisation/logs/ETS"
name = "inpi_initial_partiel_evt_new_ets_status_final_InitialPartielEVTNEW_logs.json"
for i in range(0, 7):
    s3.download_file(key= '{0}/{1}_{2}'.format(path, i, name))

In [ ]:
for i in range(0, 7):
    shutil.move("{0}/programme_matching/{1}_{2}".format(path,i, name),
            "/programme_matching/data/logs")

In [ ]:
data = []
for file in glob.glob(
    "{}/Data_preprocessed/programme_matching/data/logs/*.json".format(path)):
    with open(file) as f:
        for line in f:
            data.append(json.loads(line))

In [ ]:
logs = pd.json_normalize(data).sort_values(by = 'key')
logs

In [ ]:
logs[['total_match_rows_current']].sum()

In [ ]:
logs[['perc_total_match_rows_initial']].sum()

In [ ]:
logs[['perc_total_match_siren_initial']].sum()

In [ ]:
logs[['key',
    'perc_total_match_rows_initial',
      'perc_total_match_siren_initial']].set_index('key').plot.bar(stacked=False)

In [ ]:
logs[['total_match_rows_current']].plot.bar(stacked=False)